In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.signal import detrend
import os
import pickle

# ==============================
# Step 1: Load and Preprocess Data
# ==============================
def load_and_preprocess_data(file_path, target_column, sequence_length, scaler_type='StandardScaler', detrend_data=False):
    # Load data
    data = pd.read_csv(file_path)

    # Detrend the data if required (e.g., for Battery Level)
    if detrend_data:
        data[target_column] = detrend(data[target_column])

    # Normalize the target column
    if scaler_type == 'MinMaxScaler':
        scaler = MinMaxScaler()
    else:
        scaler = StandardScaler()

    data[target_column] = scaler.fit_transform(data[[target_column]])

    # Create sequences
    sequences = []
    for i in range(len(data) - sequence_length):
        seq = data[target_column].iloc[i:i + sequence_length].values
        sequences.append(seq)

    sequences = np.array(sequences)
    sequences = np.expand_dims(sequences, axis=2)  # Add feature dimension
    return sequences, scaler

# ==============================
# Step 2: Define LSTM Autoencoder
# ==============================
def create_lstm_autoencoder(sequence_length, input_dim, latent_dim=32):
    # Encoder
    input_layer = Input(shape=(sequence_length, input_dim))
    encoded = LSTM(latent_dim, activation='relu', return_sequences=False)(input_layer)
    encoded = Dense(latent_dim // 2, activation='relu')(encoded)

    # Decoder
    decoded = RepeatVector(sequence_length)(encoded)
    decoded = LSTM(latent_dim, activation='relu', return_sequences=True)(decoded)
    decoded = TimeDistributed(Dense(input_dim))(decoded)

    # Autoencoder model
    autoencoder = Model(inputs=input_layer, outputs=decoded)
    autoencoder.compile(optimizer='adam', loss='mae')
    return autoencoder

# ==============================
# Step 3: Train and Save Models
# ==============================
def train_and_save_model(sequences, model_path, sequence_length, input_dim, latent_dim=32, epochs=50, batch_size=32):
    # Define LSTM Autoencoder
    model = create_lstm_autoencoder(sequence_length, input_dim, latent_dim)

    # Train model with early stopping
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(sequences, sequences, epochs=epochs, batch_size=batch_size, validation_split=0.2, shuffle=True, callbacks=[early_stopping])

    # Save model
    model.save(model_path)
    print(f"Model saved to {model_path}")
    return model

# ==============================
# Step 4: Main Script
# ==============================
if __name__ == "__main__":
    # Parameters
    file_path = "C:/Users/saura/OneDrive/Desktop/data_streaming/data_generators/kaladera/dwlrdata_kaladera_(1).csv"
    sequence_length = 30         # Number of timesteps per sequence
    epochs = 50                  # Training epochs
    batch_size = 32              # Batch size
    output_dir = "C:/Users/saura/Downloads/trained_model_v2"  # Directory to save models

    # Create output directory if not exists
    os.makedirs(output_dir, exist_ok=True)

    # Depth
    print("Training model for Depth...")
    depth_sequences, depth_scaler = load_and_preprocess_data(
        file_path, target_column="Depth", sequence_length=sequence_length, scaler_type='StandardScaler'
    )
    train_and_save_model(depth_sequences, os.path.join(output_dir, "depth_model_kaladera.h5"), sequence_length, input_dim=1, latent_dim=32, epochs=epochs, batch_size=batch_size)

    # Temperature
    print("Training model for Temperature...")
    temperature_sequences, temperature_scaler = load_and_preprocess_data(
        file_path, target_column="Temperature", sequence_length=sequence_length, scaler_type='StandardScaler'
    )
    train_and_save_model(temperature_sequences, os.path.join(output_dir, "temperature_model_kaladera.h5"), sequence_length, input_dim=1, latent_dim=32, epochs=epochs, batch_size=batch_size)

    # Battery Level
    print("Training model for Battery Level...")
    battery_sequences, battery_scaler = load_and_preprocess_data(
        file_path, target_column="Battery Level", sequence_length=sequence_length, scaler_type='MinMaxScaler', detrend_data=True
    )
    train_and_save_model(battery_sequences, os.path.join(output_dir, "battery_model_kaladera.h5"), sequence_length, input_dim=1, latent_dim=64, epochs=epochs, batch_size=batch_size)

    # Save scalers for future use
    scalers = {
        "Depth": depth_scaler,
        "Temperature": temperature_scaler,
        "Battery Level": battery_scaler
    }
    scaler_path = os.path.join(output_dir, "scalers_kaladera.pkl")
    with open(scaler_path, "wb") as f:
        pickle.dump(scalers, f)
    print(f"Scalers saved to {scaler_path}")

    print("Training complete. Models and scalers are saved.")


Training model for Depth...
Epoch 1/50
730/730 [==============================] - 14s 16ms/step - loss: 0.1085 - val_loss: 0.0585
Epoch 2/50
730/730 [==============================] - 11s 15ms/step - loss: 0.0523 - val_loss: 0.0460
Epoch 3/50
730/730 [==============================] - 12s 16ms/step - loss: 0.0446 - val_loss: 0.0429
Epoch 4/50
730/730 [==============================] - 10s 14ms/step - loss: 0.0416 - val_loss: 0.0332
Epoch 5/50
730/730 [==============================] - 11s 15ms/step - loss: 0.0349 - val_loss: 0.0327
Epoch 6/50
730/730 [==============================] - 11s 15ms/step - loss: 0.0323 - val_loss: 0.0300
Epoch 7/50
730/730 [==============================] - 11s 15ms/step - loss: 0.0290 - val_loss: 0.0261
Epoch 8/50
730/730 [==============================] - 11s 15ms/step - loss: 0.0279 - val_loss: 0.0257
Epoch 9/50
730/730 [==============================] - 11s 15ms/step - loss: 0.0251 - val_loss: 0.0226
Epoch 10/50
730/730 [==============================] -

D:\anaconda\envs\anomaly_detection\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/50
730/730 [==============================] - 14s 16ms/step - loss: 0.0814 - val_loss: 0.0379
Epoch 2/50
730/730 [==============================] - 11s 15ms/step - loss: 0.0326 - val_loss: 0.0324
Epoch 3/50
730/730 [==============================] - 11s 15ms/step - loss: 0.0301 - val_loss: 0.0318
Epoch 4/50
730/730 [==============================] - 11s 15ms/step - loss: 0.0291 - val_loss: 0.0443
Epoch 5/50
730/730 [==============================] - 11s 16ms/step - loss: 0.0288 - val_loss: 0.0343
Epoch 6/50
730/730 [==============================] - 11s 15ms/step - loss: 0.0280 - val_loss: 0.0305
Epoch 7/50
730/730 [==============================] - 11s 15ms/step - loss: 0.0271 - val_loss: 0.0287
Epoch 8/50
730/730 [==============================] - 11s 15ms/step - loss: 0.0261 - val_loss: 0.0282
Epoch 9/50
730/730 [==============================] - 11s 15ms/step - loss: 0.0243 - val_loss: 0.0289
Epoch 10/50
730/730 [==============================] - 11s 16ms/step - loss: 0.021

D:\anaconda\envs\anomaly_detection\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/50
730/730 [==============================] - 41s 50ms/step - loss: 0.0437 - val_loss: 0.0164
Epoch 2/50
730/730 [==============================] - 36s 50ms/step - loss: 0.0186 - val_loss: 0.0158
Epoch 3/50
730/730 [==============================] - 37s 50ms/step - loss: 0.0158 - val_loss: 0.0203
Epoch 4/50
730/730 [==============================] - 36s 50ms/step - loss: 0.0136 - val_loss: 0.0120
Epoch 5/50
730/730 [==============================] - 36s 50ms/step - loss: 0.0097 - val_loss: 0.0094
Epoch 6/50
730/730 [==============================] - 36s 50ms/step - loss: 0.0091 - val_loss: 0.0079
Epoch 7/50
730/730 [==============================] - 36s 50ms/step - loss: 0.0072 - val_loss: 0.0061
Epoch 8/50
730/730 [==============================] - 36s 50ms/step - loss: 0.0061 - val_loss: 0.0152
Epoch 9/50
730/730 [==============================] - 36s 50ms/step - loss: 0.0063 - val_loss: 0.0050
Epoch 10/50
730/730 [==============================] - 36s 50ms/step - loss: 0.005

D:\anaconda\envs\anomaly_detection\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
